# 使用LangChain和RAGAS对RAG系统进行自动有效评估

我们主要讨论一下LLM RAG问答系统中一个重要的组成部分:

- Evaluation

我们主要使用LangChain 构建RAG问答系统，利用 RAGAS 框架进行评估，因为它正逐渐成为评估 RAG 系统的标准方法

### 首先安装 依赖

In [1]:
%pip install -U -q langchain openai ragas arxiv pymupdf chromadb wandb tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### 数据准备

主要以Arxiv的论文为例进行评估，通过 `ArxivLoader` 加载数据(论文)作为RAG的上下文。

- [`ArxivLoader`](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.arxiv.ArxivLoader.html#langchain_community.document_loaders.arxiv.ArxivLoader)


In [3]:
from langchain.document_loaders import ArxivLoader

paper_docs = ArxivLoader(query="2309.15217", load_max_docs=1).load()
len(paper_docs)

1

In [4]:
for doc in paper_docs:
  print(doc.metadata)

{'Published': '2025-04-28', 'Title': 'Ragas: Automated Evaluation of Retrieval Augmented Generation', 'Authors': 'Shahul Es, Jithin James, Luis Espinosa-Anke, Steven Schockaert', 'Summary': 'We introduce Ragas (Retrieval Augmented Generation Assessment), a framework\nfor reference-free evaluation of Retrieval Augmented Generation (RAG)\npipelines. RAG systems are composed of a retrieval and an LLM based generation\nmodule, and provide LLMs with knowledge from a reference textual database,\nwhich enables them to act as a natural language layer between a user and\ntextual databases, reducing the risk of hallucinations. Evaluating RAG\narchitectures is, however, challenging because there are several dimensions to\nconsider: the ability of the retrieval system to identify relevant and focused\ncontext passages, the ability of the LLM to exploit such passages in a faithful\nway, or the quality of the generation itself. With Ragas, we put forward a\nsuite of metrics which can be used to eval

### 创建RAG文本分割、Embedding model 、 向量库存储

我们主要使用 `RecursiveCharacterTextSplitter` 切割文本，通过`OpenAIEmbeddings()`进行文本编码，存储到 `VectorStore`。

- `RecursiveCharacterTextSplitter()`
- `OpenAIEmbeddings()`
- `Chroma`

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

docs = text_splitter.split_documents(paper_docs)

vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(base_url="..."))

C:\Users\kalin\AppData\Local\Temp\ipykernel_70320\2561186815.py:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(base_url="..."))


In [6]:
len(docs)

107

In [7]:
print(max([len(chunk.page_content) for chunk in docs]))

497


现在我们可以利用 `Chroma` 向量库的 `.as_retriever()` 方式进行检索，需要控制的主要参数为 `k`

In [8]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k" : 3})

In [9]:
relevant_docs = base_retriever.get_relevant_documents("What is Retrieval Augmented Generation?")

C:\Users\kalin\AppData\Local\Temp\ipykernel_70320\3147441568.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = base_retriever.get_relevant_documents("What is Retrieval Augmented Generation?")


In [10]:
len(relevant_docs)

3

### 创建prompt ——— 生成答案
我们需要利用`LLM`对`Context` 生成一系列的问题的`answer`


In [11]:
from langchain import PromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

Question: {question} 

Context: {context} 

Answer:
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=["context","question"]
  )

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \n\nQuestion: {question} \n\nContext: {context} \n\nAnswer:\n"


### 生成`answer`,利用LLM
利用 `Runnable` 定义一个 `chain` 实现rag全流程。

In [12]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, base_url="...")

rag_chain = (
    {"context": base_retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

C:\Users\kalin\AppData\Local\Temp\ipykernel_70320\3202603931.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, base_url="...")


#### 创建 RAGAs 所需的数据
question  Answer   contexts  ground_truths

In [37]:
# Ragas 数据集格式要求  ['question', 'answer', 'contexts', 'ground_truths']
'''
{
    "question": [], <-- 问题基于Context的
    "answer": [], <-- 答案基于LLM生成的
    "contexts": [], <-- context
    "ground_truths": [] <-- 标准答案
}
'''

from datasets import Dataset

questions = ["What is faithfulness ?", 
             "How many pages are included in the WikiEval dataset, and which years do they cover information from?",
             "Why is evaluating Retrieval Augmented Generation (RAG) systems challenging?",
            ]
ground_truths = ["Faithfulness refers to the idea that the answer should be grounded in the given context.",
                 " To construct the dataset, we first selected 50 Wikipedia pages covering events that have happened since the start of 2022.",
                "Evaluating RAG architectures is, however, challenging because there are several dimensions to consider: the ability of the retrieval system to identify relevant and focused context passages, the ability of the LLM to exploit such passages in a faithful way, or the quality of the generation itself."]
answers = []
contexts = []

# 生成答案
for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in base_retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    # "ground_truths": ground_truths,
    "reference": ground_truths,
}
dataset = Dataset.from_dict(data)


In [38]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 3
})

In [39]:
%pip show ragas

Name: ragas
Version: 0.2.15
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: d:\miniconda3\envs\rag\lib\site-packages
Requires: appdirs, datasets, diskcache, langchain, langchain-community, langchain-core, langchain_openai, nest-asyncio, numpy, openai, pydantic, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### 使用RAGAs 进行评估

In [40]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    # context_relevancy,
    context_recall,
    context_precision,
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

ragas_llm = LangchainLLMWrapper(llm)
embedding = OpenAIEmbeddings(base_url="...")
ragas_embedding = LangchainEmbeddingsWrapper(embedding)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    llm=ragas_llm,
    embeddings=ragas_embedding
)

result

Evaluating: 100%|██████████| 12/12 [00:26<00:00,  2.19s/it]


{'context_precision': 0.9444, 'context_recall': 1.0000, 'faithfulness': 0.8000, 'answer_relevancy': 0.9275}

In [41]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

df = result.to_pandas()
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What is faithfulness ?,"[First, Faithfulness refers to the idea that the an-\nswer should be grounded in the given context. This\nis important to avoid hallucinations, and to ensure\nthat the retrieved context can act as a justification\nfor the generated answer. Indeed, RAG systems are\noften used in applications where the factual con-\nsistency of the generated text w.r.t. the grounded\nsources is highly important, e.g. in domains such as\nlaw, where information is constantly evolving. Sec-, Faithfulness\nTo obtain human judgements about\nfaithfulness, we first used ChatGPT to answer the\nquestion without access to any additional context.\nWe then asked the annotators to judge which of the\ntwo answers was the most faithful (i.e. the standard\none or the one generated without context), given\nthe question and corresponding Wikipedia page.\nAnswer relevance\nWe first used ChatGPT to\nobtain candidate answers with lower answer rel-\nevance, using the following prompt:, an answer as(q). When building a RAG system,\nwe usually do not have access to human-annotated\ndatasets or reference answers. We therefore fo-\ncus on metrics that are fully self-contained and\nreference-free. We focus in particular three quality\naspects, which we argue are of central importance.\nFirst, Faithfulness refers to the idea that the an-\nswer should be grounded in the given context. This\nis important to avoid hallucinations, and to ensure]",Faithfulness refers to the idea that the answer should be grounded in the given context. It is important to avoid hallucinations and ensure that the retrieved context can act as a justification for the generated answer.,Faithfulness refers to the idea that the answer should be grounded in the given context.,0.833333,1.0,1.0,0.852842
1,"How many pages are included in the WikiEval dataset, and which years do they cover information from?","[which we refer to as WikiEval4. To construct the\ndataset, we first selected 50 Wikipedia pages cov-\nering events that have happened since the start of\n20225. In selecting these pages, we prioritised\nthose with recent edits. For each of the 50 pages,\nwe then asked ChatGPT to suggest a question that\ncan be answered based on the introductory section\nof the page, using the following prompt:\nYour task is to formulate a question from\ngiven context satisfying the rules given\nbelow:, which are annotated with human judgments. We\ncan then verify to what extent our metrics agree\nwith human assessments of faithfulness, answer\nrelevance and context relevance. Since we are not\naware of any publicly available datasets that could\nbe used for this purpose, we created a new dataset,\nwhich we refer to as WikiEval4. To construct the\ndataset, we first selected 50 Wikipedia pages cov-\nering events that have happened since the start of\n20225. In selecting these pages, we prioritised, cessing (EMNLP-IJCNLP), pages 563–578, Hong\nKong, China. Association for Computational Lin-\nguistics.\nA\nExamples from WikiEval\nTables 2, 3 and 4 show examples from the WikiEval\ndataset, focusing in particular on answers with high\nand low faithfulness (Table 2), high and low answer\nrelevance (Table 3), and high and low context rele-\nvance (Table 4).\nQuestion\nContext\nAnswer\nWho directed the film Op-\npenheimer and who stars\nas J. Robert Oppenheimer\nin the film?]",The WikiEval dataset includes 50 pages covering events that have happened since the start of 2025.,"To construct the dataset, we first selected 50 Wikipedia pages covering events that have happened since the start of 2022.",1.000000,1.0,1.0,0.929789
2,Why is evaluating Retrieval Augmented Generation (RAG) systems challenging?,"[retrieval-augmented systems is thus paramount. In\npractice, RAG systems are often evaluated in terms\nof the language modelling task itself, i.e. by mea-\nsuring perplexity on some reference c